In [ ]:
import camelot
import glob
import pandas as pd
import re
import numpy as np
from datetime import datetime

# Extracting Statements from SimplyGo

1. Log in to (SimplyGo)[https://simplygo.transitlink.com.sg/].
2. Download monthly statements from the [My Statements](https://simplygo.transitlink.com.sg/Cards/Transactions) tab.
3. Place the PDF(s) into the same directory as the notebook.

# Code

## Convert PDF to Dataframes using Camelot

In [ ]:
pdf_paths = glob.glob("*.pdf")
print(f'{len(pdf_paths)} paths found... {pdf_paths}')

In [ ]:
def get_all_dataframes(paths):
    dfs = []
    for path in paths:
        tables = camelot.read_pdf(path, pages='1-end', flavor='stream', edge_tol=50, row_tol=20)
        for i,table in enumerate(tables):
            try:
                _df = table.df
                dfs.append(_df)
            except:
                print(f"Error for table {i} in {path}...")
    return dfs

In [ ]:
dfs = get_all_dataframes(pdf_paths)

## Data Transformation

In [ ]:
cleaned_dfs = []
for df in dfs:
    if df.iloc[0,0] == "TRAVEL / PAYMENT HISTORY":
        df = df.iloc[2:]
    df = df[[0,1,2]].rename(columns={0:'datetime',1:'detail',2:'charges'})
    cleaned_dfs.append(df)

df = pd.concat(cleaned_dfs)
df = df[df.charges.str.match(r'\$[0-9]+\.[0-9]+')]
df = df[df.datetime != 'POSTED'].reset_index(drop=True)

In [ ]:
curr_date = ""
for i,row in df.iterrows():
    if re.match(r"^[A-Z][a-z]{2}, \d{2}/\d{2}/\d{4}$",row['datetime']):
        curr_date = row['datetime']
    else:
        df.loc[i,'datetime'] = f"{curr_date}, {row['datetime']}"

df = df[~df.datetime.str.match(r"^[A-Z][a-z]{2}, \d{2}/\d{2}/\d{4}$")]
df['datetime'] = pd.to_datetime(df['datetime'], format='%a, %d/%m/%Y, %I:%M %p')
df['day'] = df['datetime'].dt.day
df['month'] = df['datetime'].dt.month
df['year'] = df['datetime'].dt.year
df['hour'] = df['datetime'].dt.hour
df['minute'] = df['datetime'].dt.minute
df['dow'] = df['datetime'].dt.day_name()

df['detail'] = df['detail'].str.replace('\n',' ')
df[['origin', 'destination']] = df['detail'].str.split(' - ',n=1, expand=True)
df['charges'] = df['charges'].str.replace("$","").astype(float)

for i,row in df.iterrows():
    r = re.search(r"\((\d+[a-zA-z]?)\)",row['destination'])
    if r is not None:
        df.loc[i,'mode'] = 'bus'
        df.loc[i,'bus_no'] = r.group(1)
        df.loc[i,'destination'] = re.sub(r"\((\d+[a-zA-z]?)\)", "", row['destination'])
    else:
        df.loc[i,'origin'] = row['origin'] + " Station"
        df.loc[i,'destination'] = row['destination'] + " Station"
        df.loc[i,'mode'] = 'train'

# Save Output

In [ ]:
df

In [ ]:
today = datetime.today().strftime("%Y-%m-%d")
df.to_csv(f'public_trasport_record_{today}.csv',index=False)